Based on news20textclassification.ipynb

https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html
https://richliao.github.io/supervised/classification/2016/11/26/textclassifier-convolutional/

In [0]:
import os
import sys
import keras
from keras import utils as np_utils
import numpy as np
from keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense
from keras import Model
import pandas as pd

In [0]:
#os.mkdir("news20")

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


Get news20 and twitter

In [0]:
#!wget http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz 

In [0]:
!wget http://nlp.stanford.edu/data/glove.twitter.27B.zip 

--2019-09-08 22:41:11--  http://nlp.stanford.edu/data/glove.twitter.27B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.twitter.27B.zip [following]
--2019-09-08 22:41:11--  https://nlp.stanford.edu/data/glove.twitter.27B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip [following]
--2019-09-08 22:41:11--  http://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1520408563 (1.4G) [appli

In [0]:
# import tarfile
# tar = tarfile.open("news20.tar.gz")
# tar.extractall("news20")
# tar.close()

In [0]:
import zipfile

zip_ref = zipfile.ZipFile("glove.twitter.27B.zip", 'r')
zip_ref.extractall("/content/gdrive/My Drive/glove.twitter.27B/")
#zip_ref.extractall("")
zip_ref.close()


In [0]:
# TEXT_DATA_DIR = "news20/20_newsgroup"
# texts = []  # list of text samples
# labels_index = {}  # dictionary mapping label name to numeric id
# labels = []  # list of label ids
# for name in sorted(os.listdir(TEXT_DATA_DIR)):
#     path = os.path.join(TEXT_DATA_DIR, name)
#     if os.path.isdir(path):
#         label_id = len(labels_index)
#         labels_index[name] = label_id
#         for fname in sorted(os.listdir(path)):
#             if fname.isdigit():
#                 fpath = os.path.join(path, fname)
#                 if sys.version_info < (3,):
#                     f = open(fpath)
#                 else:
#                     f = open(fpath, encoding='latin-1')
#                 t = f.read()
#                 i = t.find('\n\n')  # skip header
#                 if 0 < i:
#                     t = t[i:]
#                 texts.append(t)
#                 f.close()
#                 labels.append(label_id)

# print('Found %s texts.' % len(texts))

Found 19997 texts.


In [0]:
df_pages_all = pd.read_pickle("/content/gdrive/My Drive/df_pages_all_backup.pkl")
#Filter out nulls
df = df_pages_all[df_pages_all.initial_message_text.notnull() == True]

from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)



In [0]:
#texts = train.initial_message_text.to_list()
texts = list([])
for i in range(len(train.initial_message_text)):
  texts.append(train.initial_message_text.iloc[i].replace('\n','').replace('\t','').replace('\r',''))
texts[0]

labels = train.section_number.to_list()

In [119]:
MAX_NB_WORDS = 10000
MAX_SEQUENCE_LENGTH = 250
VALIDATION_SPLIT = .8

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = np_utils.to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 92185 unique tokens.
Shape of data tensor: (32446, 250)
Shape of label tensor: (32446, 7)


In [11]:
embeddings_index = {}
f = open("/content/gdrive/My Drive/glove.twitter.27B/glove.twitter.27B.100d.txt")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 1193514 word vectors.


In [0]:
EMBEDDING_DIM = 100

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [0]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

Note: changed max pooling 3 (global) to 5 from 35. Not sure why 35 was there.

In [0]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 3, activation='relu')(embedded_sequences)
x = MaxPooling1D(3)(x)
x = Conv1D(128, 3, activation='relu')(x)
x = MaxPooling1D(3)(x)
x = Conv1D(128, 3, activation='relu')(x)
x = MaxPooling1D(24)(x)  # global max pooling. Switched from 35
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(7, activation='softmax')(x) # Replaced len(labels_index) with 7

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])



In [123]:
# happy learning!
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=4, batch_size=128)

Train on 6490 samples, validate on 25956 samples
Epoch 1/4
6490/6490 [==============================] - 32s 5ms/step - loss: 1.6992 - acc: 0.3006 - val_loss: 1.6639 - val_acc: 0.3349
Epoch 2/4
6490/6490 [==============================] - 31s 5ms/step - loss: 1.4959 - acc: 0.4196 - val_loss: 1.4043 - val_acc: 0.4676
Epoch 3/4
6490/6490 [==============================] - 32s 5ms/step - loss: 1.3920 - acc: 0.4670 - val_loss: 1.3434 - val_acc: 0.4791
Epoch 4/4
6490/6490 [==============================] - 31s 5ms/step - loss: 1.2912 - acc: 0.5022 - val_loss: 1.3397 - val_acc: 0.4752


Single text evaluation

In [63]:
texts_evaluation_single = list(["I want to sell my 5150. It is in good condition.  I have owned it for a long time."])
texts_evaluation_single = test.initial_message_text.iloc[1050]
sequence_evaluation_single = tokenizer.texts_to_sequences(texts_evaluation_single)
data_evaluation_single = pad_sequences(sequence_evaluation_single, maxlen=MAX_SEQUENCE_LENGTH)
predictions_single = model.predict(data_evaluation_single)
print('0MainPrediction:' , str(round(predictions_single[0][0],2)),
                                         '\n1BackstagePrediction', str(round(predictions_single[0][1],2)),
                                         '\n2FOHPrediction', str(round(predictions_single[0][2],2)),
                                         '\n3Practice_RoomPrediction', str(round(predictions_single[0][3],2)),
                                         '\n4BacklinePrediction', str(round(predictions_single[0][4],2)),
                                         '\n5Merch_StandPrediction', str(round(predictions_single[0][5],2)),
                                         '\n6BarPrediction', str(round(predictions_single[0][6],2)))

0MainPrediction: 0.19 
1BackstagePrediction 0.08 
2FOHPrediction 0.12 
3Practice_RoomPrediction 0.13 
4BacklinePrediction 0.13 
5Merch_StandPrediction 0.05 
6BarPrediction 0.29


In [129]:
len(test.initial_message_text)

8112

In [0]:
texts_test = list([])
for i in range(len(test.initial_message_text)):
  texts_test.append(test.initial_message_text.iloc[i].replace('\n','').replace('\t','').replace('\r',''))

#sequence_evaluation = tokenizer.texts_to_sequences(test['initial_message_text'])
sequence_evaluation = tokenizer.texts_to_sequences(texts_test)
data_evaluation = pad_sequences(sequence_evaluation, maxlen=MAX_SEQUENCE_LENGTH)
predictions = model.predict(data_evaluation)


In [131]:
predictions_df = pd.DataFrame(data = predictions,
                              columns = ['0MainPrediction',
                                         '1BackstagePrediction',
                                         '2FOHPrediction',
                                         '3Practice_RoomPrediction',
                                         '4BacklinePrediction',
                                         '5Merch_StandPrediction',
                                         '6BarPrediction']).round(4)
predictions_df['Max'] = predictions_df.idxmax(axis=1)
predictions_df['Actual'] = test['section_number']
predictions_df['Actual'] = predictions_df['Actual'].astype(str)
predictions_df['pred'] = predictions_df['Max'].astype(str).str[0]
#predictions_df['accurate'] = 0
#predictions_df.accurate[(predictions_df.Actual) == (predictions_df.pred)] = 1
predictions_df['accurate'] = np.where((predictions_df['Actual'])==  predictions_df['pred'], 1, 0)
# for i in range(len(predictions_df)):
#   predictions_df['pred'][i] =  predictions_df.Max[i][:1]
predictions_df.head()

,0MainPrediction,1BackstagePrediction,2FOHPrediction,3Practice_RoomPrediction,4BacklinePrediction,5Merch_StandPrediction,6BarPrediction,Max,Actual,pred,accurate
0,0.2407,0.0078,0.0787,0.3460,0.0352,0.0143,0.2772,3Practice_RoomPrediction,nan,3,0
1,0.1799,0.0040,0.0885,0.0306,0.3921,0.1242,0.1808,4BacklinePrediction,3,4,0
2,0.0676,0.0027,0.1235,0.4666,0.1681,0.1653,0.0062,3Practice_RoomPrediction,3,3,1
3,0.1372,0.0027,0.1838,0.4015,0.1323,0.0979,0.0446,3Practice_RoomPrediction,nan,3,0
4,0.2482,0.0161,0.0942,0.0832,0.2918,0.0418,0.2248,4BacklinePrediction,3,4,0


In [132]:
predictions_df.accurate.value_counts()

0    7744
1     368
Name: accurate, dtype: int64